In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
import os

In [2]:
load_dotenv()
AK=os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    google_api_key=AK
)


In [3]:
class JokeState(TypedDict):
    topic:str
    joke:str
    explanation:str
    


In [4]:
def genrate_joke(state: JokeState):
    prompt=f'gnrate a joke on topic {state["topic"]}'
    response=llm.invoke(prompt).content
    return {"joke":response}

In [5]:
def genrate_explanation(state:JokeState):
    prompt=f'write explanation for joke {state["joke"]}'
    response=llm.invoke(prompt).content
    return {"explanation":response}

In [6]:
graph=StateGraph(JokeState)

graph.add_node('genrate_joke',genrate_joke)
graph.add_node('genrate_explanation',genrate_explanation)

graph.add_edge(START,'genrate_joke')
graph.add_edge('genrate_joke','genrate_explanation')
graph.add_edge('genrate_explanation',END)

In [7]:
checkpointer=InMemorySaver()
workflow=graph.compile(checkpointer=checkpointer)